# Autocfg Version Control

This notebook will explain the version control features of `autocfg` to ensure configurations can be managed properly in a long lasting project, with forward and backward compatibilities.

In [1]:
from autocfg import dataclass, field  # drop-in replacement of dataclass decorator out of dataclasses
# this time we import a AnnotateField to help mark fields in dataclasses
from autocfg import AnnotateField as AF

### Adding version annatations

Let's reuse the configuration classes in `basic.ipynb`, this time we add some version semantics to help marking fields with different purposes.

In [2]:
@dataclass(version='0.1')
class TrainConfig:
  batch_size : int = 32
  learning_rate : float = 1e-3
  lr : AF(float, deprecated='0.1') = 1e-3
  weight_decay : AF(float, added='0.1') = 1e-5

AF(AnnotateField) supports three optional markers, `added`, `deprecated`, and `deleted`, together with class-level version marker `version='0.1'`, it can provide useful validators and checkers to ensure config is valid and up to date.

In [3]:
cfg = TrainConfig()

### Deprecated field

A deprecated field works as-is except that accessing it will throw a `UserWarning` indicate that this field has been deprecated

In [4]:
print('lr:', cfg.lr)
print(cfg)

lr: 0.001
TrainConfig(batch_size=32, learning_rate=0.001, lr=0.001, weight_decay=1e-05)


/Users/zhiz/Dev/autocfg/autocfg/dataclasses.py:103: UserWarning: `<class '__main__.TrainConfig'>.lr` is deprecated in 0.1 and will be deleted in 999.0, current is 0.1
  warnings.warn(msg)


### Deleted field

Deleted field is excluded in configs, if the code-level version marker is larger than the annotation, for example, a newer version of code which completely removes the use of `lr`

In [5]:
@dataclass(version='0.3')
class NewerTrainConfig:
  batch_size : int = 32
  learning_rate : float = 1e-3
  lr : AF(float, deprecated='0.1', deleted='0.3') = 1e-3
  weight_decay : AF(float, added='0.1') = 1e-5
    
new_cfg = NewerTrainConfig()
print(new_cfg)

NewerTrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05)


It's perfectly fine and normal to instantiate the newer version config, except that the `lr` attribute disappears, `KeyError` will raise when you try to access it explicitly

In [6]:
try:
    print(new_cfg.lr)
except KeyError as e:
    print(e)

"`<class '__main__.NewerTrainConfig'>.lr` is deleted in 0.3 in <class '__main__.NewerTrainConfig'>, current is 0.3"


with such behavior, loading configuration files saved long time ago is no longer a issue even if some fields are not needed.

In [7]:
cfg.save('old_cfg.yaml')
!cat old_cfg.yaml
new_cfg = NewerTrainConfig.load('old_cfg.yaml')
print('cfg loaded in newer version:\n', new_cfg)

# TrainConfig
batch_size: 32
learning_rate: 0.001
lr: 0.001
weight_decay: 1.0e-05
cfg loaded in newer version:
 NewerTrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05)


### Added field

This is a experimental feature and is under development, `added` marker is only served as documentatary purposes.

## Conclusion

To ensure we can track the changes of hyper-parameter fields, never delete fields in the configuration class, use `AnnotateField` instead.

`autocfg` will ensure deprecated/deleted fields can be properly handled without extra effort.